In [1]:
#### analysis of animal shelter data ###
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Load the data
# source: 
# https://www.kaggle.com/datasets/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes?resource=download

data = pd.read_csv('aac_intakes_outcomes.csv')

df = pd.DataFrame(data)
#df.head(20)
df.columns

#len(df)
#79672

#len(df.columns)
#41

Index(['age_upon_outcome', 'animal_id_outcome', 'date_of_birth',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome',
       'age_upon_outcome_(days)', 'age_upon_outcome_(years)',
       'age_upon_outcome_age_group', 'outcome_datetime', 'outcome_month',
       'outcome_year', 'outcome_monthyear', 'outcome_weekday', 'outcome_hour',
       'outcome_number', 'dob_year', 'dob_month', 'dob_monthyear',
       'age_upon_intake', 'animal_id_intake', 'animal_type', 'breed', 'color',
       'found_location', 'intake_condition', 'intake_type', 'sex_upon_intake',
       'count', 'age_upon_intake_(days)', 'age_upon_intake_(years)',
       'age_upon_intake_age_group', 'intake_datetime', 'intake_month',
       'intake_year', 'intake_monthyear', 'intake_weekday', 'intake_hour',
       'intake_number', 'time_in_shelter', 'time_in_shelter_days'],
      dtype='object')

In [2]:
## data cleaning

# convert cols with dates to datetime format
for col in ['date_of_birth', 'outcome_datetime', 'outcome_year', 'outcome_month', 'dob_year', 'dob_month', 'dob_monthyear', 'intake_datetime', 'intake_month', 'intake_year', 'intake_monthyear']:
   df[col] = pd.to_datetime(df[col], format='mixed')


DateParseError: month must be in 1..12: 1998--19 00:00:00, at position 19

In [ ]:
# check for NaNs
print('columns with "NaN" strings')
print('----------')

nan_counts = df.isna().sum()

# print the result
print(nan_counts)


In [4]:
# check for missing values
#print('missing values:')
#print('---------')
#print(df.isnull().sum())


In [ ]:
df.outcome_subtype.unique()
df.outcome_type.unique()

# replace null values in the outcome_subtype column with 'unknown'
df.outcome_subtype.fillna('unknown', inplace=True)

# verifying the outcome subtypes
print(df.outcome_subtype.unique())

# replace null values in outcome_type column with 'Unknown'
df.outcome_type.fillna('unknown', inplace=True)

# verifying the outcome types
print(df.outcome_type.unique())


In [6]:
### possible questions:
## dogs
# length of stay/outcome type by breed/colour/age 
# (more positive for breeds such as labs? more negative for black animals?)
# euthanasia rates over time (fewer nowadays?)
# intake type over time (more owner surrender?)

In [ ]:
# test if column 'animal_id_outcome' is the same as 'animal_id_intake'
df.head(3)
df['animal_id_intake'].equals(df['animal_id_outcome'])  # Returns True

In [ ]:
## filtering and subsetting
# generate a subset of the dataframe to address specific questions

df_analysis = df[['animal_id_intake', 'animal_type', 'intake_type', 'intake_condition', 'sex_upon_intake', 'age_upon_intake_(years)', 'age_upon_intake_age_group','breed', 'color', 'time_in_shelter_days', 'outcome_subtype', 'intake_monthyear', 'outcome_monthyear', 'outcome_year', 'outcome_type', 'count']]
df_analysis.columns


In [ ]:
# generate dog df
df_dogs = df_analysis[df_analysis['animal_type']=='Dog']
len(df_dogs)

# check if it's indeed only dogs
df_dogs['animal_type'].unique()


In [ ]:
# count number of unique dogs total
tot_dogs = df_dogs['animal_id_intake'].unique()

print(len(tot_dogs))
#38909 dogs

In [ ]:
# number of dogs per intake_type
df_dogs.head()

dogs_per_intake_type = df_dogs.groupby('intake_type').count().reset_index()
dogs_per_intake_type



In [ ]:
# plot
plt.figure(figsize=(10,4))
sns.barplot(x='intake_type',y='count', data = dogs_per_intake_type)
plt.xlabel('intake type')
plt.ylabel('count')
plt.tight_layout()
plt.show()


In [13]:
# count of dogs per age_upon_intake_age_group
dogs_by_age = df_dogs.groupby('age_upon_intake_(years)').count().reset_index()
#dogs_by_age


In [ ]:
# histogram of intake ages
plt.figure(figsize=(10,4))
sns.histplot(df_dogs['age_upon_intake_(years)'], bins=21)
plt.xlabel('age at intake [years]')
plt.ylabel('count')
plt.tight_layout()
plt.show()

In [ ]:
# test if a similar picture emerges if ages are grouped into age groups
# first find unique values in age group
df_analysis['age_upon_intake_age_group'].unique()

# replace these by categorical variables (or learn how to remove the brackets)
df_test = df_analysis.replace(['(7.5, 10.0]', '(5.0, 7.5]', '(15.0, 17.5]', '(12.5, 15.0]',
       '(17.5, 20.0]', '(10.0, 12.5]', '(2.5, 5.0]', '(-0.025, 2.5]',
       '(20.0, 22.5]', '(22.5, 25.0]'], ['07.5_10.0', '05.0_07.5','15.0_17.5', '12.5_15.0',
    '17.5_20.0', '10.0_12.5', '02.5_05.0','00.0_02.5','20.0_22.5','22.5_25.0'])

df_test['age_upon_intake_age_group'].unique()

# count of dogs per age_upon_intake_age_group
dogs_by_age_group = df_test.groupby('age_upon_intake_age_group').count().reset_index()
dogs_by_age_group.sort_values(by='age_upon_intake_age_group',ascending=True)

In [ ]:
# bar plot of intake age groups
plt.figure(figsize=(10,4))
sns.barplot(x='age_upon_intake_age_group', y='count', data=dogs_by_age_group)
plt.xlabel('age group at intake')
plt.ylabel('count')
plt.tight_layout()
plt.show()

In [ ]:
# scatterplot of length of stay in relation to age
plt.figure(figsize=(12,6))
sns.scatterplot(x='age_upon_intake_(years)', y='time_in_shelter_days', data=df_dogs)

plt.xlabel('age at intake [years]')
plt.ylabel('length of stay')
plt.tight_layout()
plt.show()
           

In [ ]:
# filter dogs with above-average lengths of stay
avg_stay = df_dogs['time_in_shelter_days'].mean()
print(f"the average length of stay is {avg_stay.round(1)} days")

above_avg_stay = df_dogs[df_dogs['time_in_shelter_days'] > df_dogs['time_in_shelter_days'].mean()]

print(f"there are {len(above_avg_stay)} dogs that stay longer than the average")

In [ ]:
# length of stay statistics by breed/age
stay_stats_by_breed = df_dogs.groupby('breed')['time_in_shelter_days'].agg(['mean', 'min', 'max'])
stay_stats_by_breed.sort_values('mean', ascending=True).tail()


stay_stats_by_age_group = df_test.groupby('age_upon_intake_age_group')['time_in_shelter_days'].agg(['mean', 'min', 'max'])
stay_stats_by_age_group.sort_values('age_upon_intake_age_group', ascending=True).round(1)


In [ ]:
# most and least taken in breeds of dogs
top_breeds_taken_in = df_dogs['breed'].value_counts(sort=True).head()
bottom_breeds_taken_in = df_dogs['breed'].value_counts(sort=True).tail()
top_breeds_taken_in
#bottom_breeds_taken_in

In [ ]:
# add a column showing adoptions relative to total intake per given monthyear 
# Step 1: Calculate the total number of outcomes per intake_monthyear
total_outcomes = df_dogs.groupby('intake_monthyear')['count'].sum()

# Step 2: Calculate the number of adoptions per intake_monthyear
adoptions = df_dogs[df_dogs['outcome_type'] == 'Adoption'].groupby('intake_monthyear')['count'].sum()

# Step 3: Calculate the ratio of adoptions to total outcomes for each intake_monthyear
adoption_ratio = adoptions / total_outcomes

# Step 4: Add a new column to the original DataFrame
df_dogs['adoption_ratio'] = df_dogs['intake_monthyear'].map(adoption_ratio)



In [ ]:
outcomes_over_time = df_dogs.groupby(['outcome_monthyear', 'outcome_type'])['count'].sum().unstack()

# Step 2: Plot the data using a line plot
plt.figure(figsize=(10, 6))

# Step 3: Plot a line for each outcome_type
outcomes_over_time.plot(kind='line', marker='o', ax=plt.gca())

# Step 4: Customize the plot
plt.xlabel('Outcome Month/Year')
plt.ylabel('Count of Outcomes')
plt.title('Adoptions and Other Outcomes Over Time')
plt.legend(title='Outcome Type')

# Display the plot
plt.show()

In [ ]:
######## Frage an Nenad #########
# ich wollte hier einen line plot mit outcome_monthyear auf dr x achse, sortiert nach Datum (was auch gelungen ist)
# aber EINEN Wert für adoption_ratio pro gegebenem monthyear Wert. Ich verstehe nicht, wieso dort mehrere auftauchen.



plt.figure(figsize=(15,6))

#plt.plot(adoption_counts['outcome_year'].sort_values(ascending=True), adoption_counts['count'], color='black')
plt.plot(df_dogs['outcome_monthyear'].sort_values(ascending=True), df_dogs['adoption_ratio'],marker='o', linestyle='-', color='b')
plt.xlabel('year')
plt.ylabel('adoption rate')
plt.xticks(rotation = 45)

plt.tight_layout()
plt.show()

In [ ]:
# Assuming df_dogs is your original DataFrame
# Group by outcome_monthyear and calculate mean adoption ratio
monthly_adoption_ratio = df_dogs.groupby('outcome_monthyear')['adoption_ratio'].mean().reset_index()

# Sort by date
monthly_adoption_ratio['outcome_monthyear'] = pd.to_datetime(monthly_adoption_ratio['outcome_monthyear'])
monthly_adoption_ratio = monthly_adoption_ratio.sort_values('outcome_monthyear')

# Create the plot
plt.figure(figsize=(15,6))
plt.plot(monthly_adoption_ratio['outcome_monthyear'], 
         monthly_adoption_ratio['adoption_ratio'],
         marker='o', linestyle='-', color='b')

plt.xlabel('Year')
plt.ylabel('Adoption Rate')
plt.title('Monthly Adoption Rate Over Time')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

# Group by 'outcome_monthyear' and calculate the mean 'adoption_ratio'
# df_grouped = df_dogs.groupby('outcome_monthyear', as_index=False)['adoption_ratio'].max()
df_grouped = df_dogs.groupby('outcome_monthyear', as_index=False)['adoption_ratio'].mean()
# df_grouped = df_dogs.groupby('outcome_monthyear', as_index=False)['adoption_ratio'].min()

# Sort by 'outcome_monthyear'
df_grouped = df_grouped.sort_values('outcome_monthyear', ascending=True)

# Plot the aggregated data
plt.figure(figsize=(15,6))
plt.plot(df_grouped['outcome_monthyear'], df_grouped['adoption_ratio'], marker='o', linestyle='-', color='b')
plt.xlabel('year')
plt.ylabel('adoption rate')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()
df_grouped['adoption_ratio']
# df_grouped['outcome_monthyear']